In [1]:
import numpy as np

In [2]:
import json_lines as jl

In [3]:
import pandas as pd

In [4]:
import re
noaq = re.compile('\?$')
noacom = re.compile('\.{3}$')

In [5]:
raw_data = []
with open('i.jl', 'r') as f:
    for item in jl.reader(f):
        raw_data.append(item)

In [6]:
topics = {}
for i in raw_data:
    for t in i['topic']:
        if t not in topics:
            topics[t] = 1
        else:
            topics[t] += 1

In [7]:
for i in sorted(topics, key=topics.get, reverse=True)[:50]:
    print(i+': ', str(topics[i]))

Career Advice:  6175
Exams and Tests:  5390
Dating and Relationships:  3989
College Advice:  3726
Life Advice:  3678
India:  3454
Medicine and Healthcare:  3448
Jobs and Careers:  3327
Interpersonal Interaction:  3287
Dating Advice:  2954
Human Behavior:  2869
Life and Living:  2665
Physics:  2591
Computer Programming:  2583
Personal Question:  2532
Politics of the United States of America:  2525
Education:  2457
The United States of America:  2260
Religion:  2136
Philosophy of Everyday Life:  2030
Relationship Advice:  1922
Mathematics:  1809
Health:  1678
Donald Trump (politician, business person):  1647
Colleges and Universities:  1581
Understanding Human Behavior:  1535
Quora:  1515
Joint Entrance Examination (JEE):  1507
Food:  1463
Engineering:  1448
Money:  1420
Investing:  1383
Chemistry:  1383
Cars and Automobiles:  1381
Hypothetical Scenarios:  1380
Psychology of Everyday Life:  1359
Music:  1350
Love:  1332
History:  1329
Books:  1312
Movies:  1311
Web Development:  1295
Eng

In [8]:
raw_data[0]

{'a': ["Carly, I thank you for the A2A but I failed badly. No clue. Guess I could have researched it but that isn't fair. Nope, nada. Am embarrassed, I guess am out of the club now.",
  "Carly, I thank you for the A2A but I failed badly. No clue. Guess I could have researched it but that isn't fair. Nope, nada. Am embarrassed, I guess am out of the club now."],
 'q': ['Do you remember mistaking ripjaws from Ben 10 for a mermaid?'],
 'topic': ['Ripjaws',
  'Ben 10 (creative franchise)',
  'Mermaids',
  'Childhood Memories',
  'Cartoons']}

In [9]:
def cleanup(data):
    arr_e = []
    n = 0
    # data = list(set(data))
    for i in data:
        try:
            i['q'] = i['q'][0]
        except:
            arr_e.append(n)
            continue
        arr = []
        for a in i['a']:
            try:
                if a!='\n' and a[-1]!='?' and a[-3]!='.' and a!='Footnotes':
                    arr.append(a)
                else:
                    pass
            except:
                pass
        i['a'] = list(set(arr))
        n+=1
    return data, arr_e

In [10]:
data, arr_e = cleanup(raw_data)

In [11]:
len(data)

139498

In [12]:
for i in arr_e:
    data.remove(data[i])

In [13]:
frame = pd.DataFrame(columns=['q', 'a'])

In [ ]:
c_q = 0
c = 0
for i in data:
    c_q+=1
    for n in i['a']:
        frame.loc[c] = [i['q'], n]
        #print(c)
        c+=1
    if c_q%1000==0:
        print('QUES: '+str(c_q))

In [ ]:
with open('temp.tsv', 'w') as temp:
    frame.to_csv(temp, sep='\t')

In [ ]:
frame.describe()

In [14]:
def prepare(data):
    d = {}
    c = 0
    for i in data:
        for n in i['a']:
            d[c] = {'q': i['q'], 'a': n}
            c+=1
    return d

In [15]:
todict = prepare(data)

In [16]:
frame = frame.from_dict(todict, orient='index')

In [17]:
frame.describe()

,a,q
count,1521712,1521712
unique,1057222,102542
top,Good luck!,What's the easiest way to make money online?
freq,1068,1808


In [ ]:
def get_vocabs(data):
    arr_en = []
    arr_fr = []
    count = 0
    for i in data:
        arr_en.extend(i['q'].split(' '))
        for n in i['a']:
            arr_fr.extend(n.split(' '))
        if count%100==0:
            arr_en=list(set(arr_en))
            arr_fr=list(set(arr_fr))
            print(count)
        count+=1
    return len(list(set(arr_en))), len(list(set(arr_fr)))

In [ ]:
print(get_vocabs(data))

In [ ]:
def get_q_vocab(data):
    arr_en = []
    arr_fr = []
    count = 0
    for i in data:
        arr_en.extend(i['q'].split(' '))
        if count%100==0:
            arr_en=list(set(arr_en))
            arr_fr=list(set(arr_fr))
            print(count)
        count+=1
    return list(set(arr_en))

In [ ]:
data[0]

In [ ]:
len(get_q_vocab(data))

In [ ]:
def get_a_vocab(data):
    arr_en = []
    arr_fr = []
    count = 0
    for i in data:
        for n in i['a']:
            arr_fr.extend(n.split(' '))
            if count%10000==0:
                arr_en=list(set(arr_en))
                arr_fr=list(set(arr_fr))
                print(count)
            count+=1
    return list(set(arr_fr))

In [ ]:
len(get_a_vocab(data))

In [ ]:
def get_len(s):
    return(len(s.split(' ')))

In [ ]:
frame['lenq'] = [len(i.split(' ')) for i in frame['q']]

In [ ]:
frame[:50]

In [ ]:
frame['lena'] = [len(i.split(' ')) for i in frame['a']]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
y_pred = KMeans(n_clusters=10).fit(frame[['lenq', 'lena']])

In [ ]:
[np.array(frame['lenq']), np.array(frame['lena'])]

In [ ]:
frame['class'] = y_pred.labels_

In [ ]:
frame[frame['class']==0]['q']

In [ ]:
frame = frame.drop_duplicates()

In [ ]:
for i in range(0, 9):
    print(np.median(frame[frame['class']==i]['lenq']), np.median(frame[frame['class']==i]['lena']))

In [18]:
from sklearn.model_selection import train_test_split as skspl

In [19]:
X_train, X_test, y_train, y_test = skspl(frame['q'], frame['a'], test_size=0.01)

In [22]:
en = open('big\train.en', 'w')
fr = open('big\train.fr', 'w')
for i in X_train:
    en.write(i+'\n')
en.close()
for i in y_train:
    fr.write(i+'\n')
fr.close()